### Implementation of cellpose for CAMPARI data

This notebook will read a set of .czi files (saved in green and red fluorecence).
It will convert .czi into a readable file format for cellpose.
We will run cellpose on Red fluorecence image, and plot the extracted template into the red and green flourecence image as a control. 
Later will extract cell size and intensity from green and red fluorecence for further processing.

In [5]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from cellpose import utils, io
import czifile as czifile
import pandas as pd
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here
from PIL import Image

In [6]:
### Define all relevant directories and paths
directory_path = '/home/melma31/Documents/campari/'
### Input data base with mice data
data_base_path = directory_path + 'data_base.xlsx'

### Output data base with cell data. Backup copy is done simultaneusly (this is in case the code breaks while saving data base)
data_base_path_output = directory_path + 'data_base_output.xlsx'
data_base_path_output_bkup = directory_path + 'data_base_output_bkup.xlsx'

In [12]:
### If only some animals, days, side what to be analyzed, select them in here
MOUSE = None
SIDE = None
EXAMPLE = None
DATE = 11

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]

# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images


### Parameters for CELLPOSE AND FOR POSTERIOR SELECTION
DIAMETER = 20
### MODEL SELECTION : add posibilities between model_type='cyto' or model_type='nuclei'
MODEL = 'cyto2'
CHANNELS = [0,0]
CELLSIZE = 250

In [13]:
mouse_data = pd.read_excel(data_base_path)
cell_data_output = pd.read_excel(data_base_path_output)
print(mouse_data)
#print(cell_data_output)
#mouse_number = 1 
#mouse_data = dbase.loc[dbase['mouse_id'] == mouse_number]

     date  month  year  mouse_id  condition  stimuli  side  example  \
0      11      1  2023         1        1.0      1.0   0.0      1.0   
1      11      1  2023         1        1.0      1.0   0.0      2.0   
2      11      1  2023         1        1.0      1.0   0.0      3.0   
3      11      1  2023         1        1.0      1.0   0.0      4.0   
4      11      1  2023         1        1.0      1.0   0.0      5.0   
..    ...    ...   ...       ...        ...      ...   ...      ...   
133     7      9  2022         2        0.0      0.0   1.0      3.0   
134     8      9  2022         1        NaN      NaN   NaN      NaN   
135     8      9  2022         1        NaN      NaN   NaN      NaN   
136     8      9  2022         1        NaN      NaN   NaN      NaN   
137     8      9  2022         1        NaN      NaN   NaN      NaN   

    green_file_name red_file_name  
0             GREEN           RED  
1             GREEN           RED  
2             GREEN           RED  
3  

In [16]:
### PART 2: SELECT DESIRED DATA TO ANALYSE
# Select only red channel data
selected_data = mouse_data.loc[mouse_data['date'] == DATE]
#selected_data = selected_data.loc[selected_data['mouse_id'] == MOUSE]
#selected_data = selected_data.loc[selected_data['example'] == EXAMPLE]
#selected_data = selected_data.loc[selected_data['side'] == SIDE]

#selected_data = mouse_data
mouse_data = selected_data

In [17]:
## PART 1: CONVERT ALL FILES
# load all files in the data base and convert czi to tif
for i in range(len(mouse_data)):
    date = mouse_data.iloc[i].date
    month =  mouse_data.iloc[i].month
    year =  mouse_data.iloc[i].year
    mouse = mouse_data.iloc[i].mouse_id
    folder_structure = str(date) + '.' + str(month) + '.'+ str(year) + '/animal_' + str(mouse)
    side = mouse_data.iloc[i].side
    if side == 0:
        folder_structure = folder_structure + '/left/'
    else:
        folder_structure = folder_structure + '/right/'
    example = int(mouse_data.iloc[i].example)
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + mouse_data.iloc[i].green_file_name
    images = czifile.imread(file_path + '.czi')
    images = images[0,:,:,0]
    im = Image.fromarray(images)
    im.save(file_path+'.tif')
    
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + mouse_data.iloc[i].red_file_name
    images = czifile.imread(file_path + '.czi')
    images = images[0,:,:,0]
    im = Image.fromarray(images)
    im.save(file_path+'.tif')
print('All files had been converted from .czi to .tif')

All files had been converted from .czi to .tif


In [24]:
print('Number of red channel data: ' + str(len(selected_data)))
# check that all tif files exists

# RUN CELLPOSE
from cellpose import models, io

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=False, model_type=MODEL)
channels = CHANNELS
diameter = DIAMETER


# or in a loop
for i in range(len(selected_data)):
    date = selected_data.iloc[i].date
    month =  selected_data.iloc[i].month
    year =  selected_data.iloc[i].year
    mouse = selected_data.iloc[i].mouse_id
    folder_structure = str(date) + '.' + str(month) + '.'+ str(year) + '/animal_' + str(mouse)
    side = selected_data.iloc[i].side
    if side == 0:
        folder_structure = folder_structure + '/left/'
    else:
        folder_structure = folder_structure + '/right/'
    example = int(selected_data.iloc[i].example)
    
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].red_file_name
    filesname_red = file_path
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].green_file_name
    filesname_green = file_path
    
    if not os.path.exists(filesname_red + '.tif') or not os.path.exists(filesname_green + '.tif'):
        print(filesname_red + 'does not exist')
    else:
        print(filesname_red)
        img = io.imread(filesname_red + '.tif')
        masks, flows, styles, diams = model.eval(img, diameter=diameter, channels=channels)
        # save results so you can load in gui
        #io.masks_flows_to_seg(img, masks, flows, diams, filename, channels)
        # save results as png
        #flows_list.append(flows)
        io.save_to_png(img, masks, flows, filesname_red)

Number of red channel data: 33
/home/melma31/Documents/campari/data/11.1.2023/animal_1/left/1_RED
/home/melma31/Documents/campari/data/11.1.2023/animal_1/left/2_RED
/home/melma31/Documents/campari/data/11.1.2023/animal_1/left/3_RED
/home/melma31/Documents/campari/data/11.1.2023/animal_1/left/4_RED


KeyboardInterrupt: 

In [26]:
min(img.shape)

1024

In [17]:
### THIS WILL PLOT FOR ALL SELECTED DATA orginal image and masks on top for red and green
### with red colors the ones rejected by CELLSIZE limit and in BLUE the accepted ones
from cellpose import plot
from cellpose import utils, io, transforms
from cellpose.plot import image_to_rgb, mask_overlay

# or in a loop
chan = CHANNELS
channels = CHANNELS
for i in range(len(selected_data)):
    date = selected_data.iloc[i].date
    month =  selected_data.iloc[i].month
    year =  selected_data.iloc[i].year
    mouse = selected_data.iloc[i].mouse_id
    folder_structure = str(date) + '.' + str(month) + '.'+ str(year) + '/animal_' + str(mouse)
    side = selected_data.iloc[i].side
    if side == 0:
        folder_structure = folder_structure + '/left/'
        figure_name = str(date) + '.' + str(month) + '.'+ str(year) + '_animal_' + str(mouse) + '_left_'
    else:
        folder_structure = folder_structure + '/right/'
        figure_name = str(date) + '.' + str(month) + '.'+ str(year) + '_animal_' + str(mouse) + '_right_'

    example = selected_data.iloc[i].example
    figure_name = figure_name + '_example_' + str(selected_data.iloc[i].example)
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].red_file_name
    filesname = file_path 
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].green_file_name
    filesname_2 = file_path
    mask_name = filesname + '_cp_masks.png'
    print(mask_name)
    if not os.path.exists(filesname + '.tif') or not os.path.exists(filesname_2 + '.tif') or not os.path.exists(mask_name):
        print('FILE does NOT exist')
    else:
        img = io.imread(filesname + '.tif')
        masks = io.imread(mask_name)
        img2 = io.imread(filesname_2 + '.tif')
        ncells = np.max(masks)
        print(ncells)
        fig, axes = plt.subplots(2,2)
        img0 = img.copy()
        if img0.shape[0] < 4:
            img0 = np.transpose(img0, (1,2,0))
        if img0.shape[-1] < 3 or img0.ndim < 3:
            img0 = image_to_rgb(img0, channels=channels)
        else:
            if img0.max()<=50.0:
                img0 = np.uint8(np.clip(img0*255, 0, 1))

        axes[0,0].imshow(img0)
        axes[0,0].set_title('RED CHANNEL')
        axes[0,0].axis('off')
        mask_copy = np.zeros_like(masks)
        for cell_num in range(1,ncells):
            cell_mask = np.where(masks == cell_num)
            cellsize = len(cell_mask[0])
            if cellsize >= 250:
                mask_copy[np.where(masks == cell_num)] = cell_num
        #        print(cell_num)
        #print(np.max(mask_copy))
        
        outlines = utils.masks_to_outlines(masks)
        #overlay = mask_overlay(img0, masks)
        outX, outY = np.nonzero(outlines)
        imgout= img0.copy()
        imgout[outX, outY] = np.array([255,0,0]) # pure red
        outlines = utils.masks_to_outlines(mask_copy)
        #overlay = mask_overlay(img0, masks)
        outX, outY = np.nonzero(outlines)
        imgout[outX, outY] = np.array([0,0,255]) # pure blue
        
        axes[0,1].imshow(imgout)
        axes[0,1].set_title('PREDITION')
        axes[0,1].axis('off')

        #axes[0,1].imshow(imgout)
        #axes[0,1].set_title('PREDITION')
        #axes[0,1].axis('off')

        img0 = img2.copy()
        if img0.shape[0] < 4:
            img0 = np.transpose(img0, (1,2,0))
        if img0.shape[-1] < 3 or img0.ndim < 3:
            img0 = image_to_rgb(img0, channels=channels)
        else:
            if img0.max()<=50.0:
                img0 = np.uint8(np.clip(img0*255, 0, 1))

        
        axes[1,0].imshow(img0)
        axes[1,0].set_title('GREEN CHANNEL')
        axes[1,0].axis('off')

        outlines = utils.masks_to_outlines(masks)
        #overlay = mask_overlay(img0, masks)
        outX, outY = np.nonzero(outlines)
        imgout= img0.copy()
        imgout[outX, outY] = np.array([255,0,0]) # pure red
        outlines = utils.masks_to_outlines(mask_copy)
        #overlay = mask_overlay(img0, masks)
        outX, outY = np.nonzero(outlines)
        imgout[outX, outY] = np.array([0,0,255]) # pure blue        
        
        axes[1,1].imshow(imgout)
        axes[1,1].set_title('PREDITION')
        axes[1,1].axis('off')

        plt.tight_layout()
        fig.savefig(directory_path + 'figures/'+ figure_name + '_ncells_' + str(ncells) + '.png')